In [1]:
import torch.nn as nn
import numpy as np
import torch
import yaml
from torch.autograd import Variable
from torch.utils.data import DataLoader
from txt2image_dataset import Text2ImageDataset
from utils import Utils, Logger
from PIL import Image
import os

### Dataset

In [2]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

config

{'birds_images_path': 'data/cvpr2016_cub/images/',
 'birds_embedding_path': 'data/cub_icml/',
 'birds_text_path': 'data/cvpr2016_cub/cvpr2016_cub/text_c10/',
 'birds_dataset_path': 'data/cvpr2016_cub/text2image/birds.hdf5',
 'val_split_path': 'data/cvpr2016_cub/valclasses.txt',
 'train_split_path': 'data/cvpr2016_cub/trainclasses.txt',
 'test_split_path': 'data/cvpr2016_cub/testclasses.txt',
 'flowers_images_path': 'data/cvpr2016_flowers/images/',
 'flowers_embedding_path': 'data/flowers_icml/',
 'flowers_text_path': 'data/cvpr2016_flowers/text_c10/',
 'flowers_dataset_path': 'data/cvpr2016_flowers/text2image/flowers.hdf5',
 'flowers_val_split_path': 'data/cvpr2016_flowers/valclasses.txt',
 'flowers_train_split_path': 'data/cvpr2016_flowers/trainclasses.txt',
 'flowers_test_split_path': 'data/cvpr2016_flowers/testclasses.txt'}

### Generator

In [3]:
class generator1(nn.Module):
    def __init__(self):
        super(generator1, self).__init__()
        self.image_size = 64
        self.num_channels = 3
        self.noise_dim = 100
        self.ngf = 64

        self.netG = nn.Sequential(
            nn.ConvTranspose2d(self.noise_dim, self.ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(self.ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(self.ngf * 2,self.ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(self.ngf, self.num_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (num_channels) x 64 x 64
            )

    def forward(self, z):
        return self.netG(z)

In [4]:
generator1()

generator1(
  (netG): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [5]:
class discriminator1(nn.Module):
    def __init__(self):
        super(discriminator1, self).__init__()
        self.image_size = 64
        self.num_channels = 3
        self.ndf = 64

        self.netD_1 = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(self.num_channels, self.ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(self.ndf, self.ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(self.ndf * 2, self.ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(self.ndf * 4, self.ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 8),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.netD_2 = nn.Sequential(
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(self.ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            )

    def forward(self, inp):
        x_intermediate = self.netD_1(inp)
        output = self.netD_2(x_intermediate)
        return output.view(-1, 1).squeeze(1), x_intermediate

In [6]:
discriminator1()

discriminator1(
  (netD_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (netD_2): Sequential(
    (0): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): Sigmoid()
  )
)

In [12]:
noise_dim = 100
batch_size = 64
num_workers = 1
lr = 0.0002
beta1 = 0.5
num_epochs = 100
l1_coef = 50
l2_coef = 200

vis_screen = 'gan'
checkpoints_path = 'checkpoints'
save_path = 'results'
logger = Logger(vis_screen)

Setting up a new session...


In [13]:
criterion = nn.BCELoss()
l2_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

In [14]:
# generator
generator = generator1().cuda()
generator.apply(Utils.weights_init)

# discriminator
discriminator = discriminator1().cuda()
discriminator.apply(Utils.weights_init)

optimD = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
optimG = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))


In [15]:
def train(ds='flowers'):

    if ds == 'birds':
        dataset = Text2ImageDataset(config['birds_dataset_path'], split=0)
    if ds == 'flowers':
        dataset = Text2ImageDataset(config['flowers_dataset_path'], split=0)

    data_loader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True)

    iteration = 0
    
    print("num iterations", len(data_loader), 'batch_size', batch_size)
    
    for epoch in range(num_epochs):
        for sample in data_loader:
            iteration += 1
            right_images = sample['right_images']
            # print(f"right_images shape {right_images.shape}")

            right_images = Variable(right_images.float()).cuda()

            real_labels = torch.ones(right_images.size(0))
            fake_labels = torch.zeros(right_images.size(0))

            smoothed_real_labels = torch.FloatTensor(Utils.smooth_label(real_labels.numpy(), -0.1))

            real_labels = Variable(real_labels).cuda()
            smoothed_real_labels = Variable(smoothed_real_labels).cuda()
            fake_labels = Variable(fake_labels).cuda()

            # Train the discriminator
            discriminator.zero_grad()
            outputs, activation_real = discriminator(right_images)
            # print(f"outputs shape {outputs.shape}, activation shape {activation_real.shape}")
            real_loss = criterion(outputs, smoothed_real_labels)
            real_score = outputs

            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)
            # print(f"noise shape {noise.shape}")
            fake_images = generator(noise)
            # print(f"fake_images shape {fake_images.shape}")
            outputs, _ = discriminator(fake_images)
            fake_loss = criterion(outputs, fake_labels)
            fake_score = outputs

            d_loss = real_loss + fake_loss

            d_loss.backward()
            optimD.step()

            # Train the generator
            generator.zero_grad()

            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)

            fake_images = generator(noise)

            outputs, activation_fake = discriminator(fake_images)
            _, activation_real = discriminator(right_images)

            activation_fake = torch.mean(activation_fake, 0)
            activation_real = torch.mean(activation_real, 0)

            g_loss = criterion(outputs, real_labels) \
                     + l2_coef * l2_loss(activation_fake, activation_real.detach()) \
                     + l1_coef * l1_loss(fake_images, right_images)

            g_loss.backward()
            optimG.step()
            
            if iteration % 5 == 0:
                logger.log_iteration_gan(epoch, d_loss, g_loss, real_score, fake_score)
                logger.draw(right_images, fake_images)

        logger.plot_epoch_w_scores(iteration)
        
        if epoch % 50 == 0:
            Utils.save_checkpoint(discriminator, generator, save_path, checkpoints_path, epoch)



In [ ]:
train()

num iterations 460 batch_size 64
Epoch: 0, d_loss= 0.904000, g_loss= 41.104202, D(X)= 0.730098, D(G(X))= 0.216479
Epoch: 0, d_loss= 0.832878, g_loss= 38.801735, D(X)= 0.830919, D(G(X))= 0.193332
Epoch: 0, d_loss= 1.991587, g_loss= 42.702164, D(X)= 0.958446, D(G(X))= 0.688626
Epoch: 0, d_loss= 0.787980, g_loss= 36.566078, D(X)= 0.783049, D(G(X))= 0.193160
Epoch: 0, d_loss= 0.799091, g_loss= 38.553001, D(X)= 0.849763, D(G(X))= 0.269631
Epoch: 0, d_loss= 0.459942, g_loss= 36.782024, D(X)= 0.910686, D(G(X))= 0.048683
Epoch: 0, d_loss= 0.509546, g_loss= 34.582935, D(X)= 0.713621, D(G(X))= 0.023657
Epoch: 0, d_loss= 1.754358, g_loss= 29.498199, D(X)= 0.247426, D(G(X))= 0.006536
Epoch: 0, d_loss= 0.564140, g_loss= 32.963844, D(X)= 0.737231, D(G(X))= 0.043185
Epoch: 0, d_loss= 2.022060, g_loss= 36.035110, D(X)= 0.270289, D(G(X))= 0.000373
Epoch: 0, d_loss= 0.575429, g_loss= 26.284527, D(X)= 0.837952, D(G(X))= 0.118937
Epoch: 0, d_loss= 1.124533, g_loss= 28.598059, D(X)= 0.748123, D(G(X))= 0.38

Epoch: 1, d_loss= 0.663849, g_loss= 27.028885, D(X)= 0.708902, D(G(X))= 0.157840
Epoch: 1, d_loss= 0.892843, g_loss= 26.882004, D(X)= 0.497398, D(G(X))= 0.087728
Epoch: 1, d_loss= 0.795777, g_loss= 27.950693, D(X)= 0.622509, D(G(X))= 0.184907
Epoch: 1, d_loss= 1.385639, g_loss= 28.165071, D(X)= 0.300139, D(G(X))= 0.030538
Epoch: 1, d_loss= 0.768807, g_loss= 29.806326, D(X)= 0.755765, D(G(X))= 0.275999
Epoch: 1, d_loss= 0.777480, g_loss= 30.701485, D(X)= 0.693486, D(G(X))= 0.240093
Epoch: 1, d_loss= 0.683099, g_loss= 28.040232, D(X)= 0.720249, D(G(X))= 0.183088
Epoch: 1, d_loss= 0.775969, g_loss= 28.312813, D(X)= 0.655260, D(G(X))= 0.203001
Epoch: 1, d_loss= 1.861318, g_loss= 27.189600, D(X)= 0.164112, D(G(X))= 0.018172
Epoch: 1, d_loss= 0.828105, g_loss= 28.759342, D(X)= 0.835875, D(G(X))= 0.318992
Epoch: 1, d_loss= 1.236722, g_loss= 29.771816, D(X)= 0.909757, D(G(X))= 0.518401
Epoch: 1, d_loss= 0.895714, g_loss= 29.380396, D(X)= 0.864870, D(G(X))= 0.371344
Epoch: 1, d_loss= 0.988693, 

Epoch: 2, d_loss= 0.928098, g_loss= 27.453526, D(X)= 0.579564, D(G(X))= 0.230442
Epoch: 2, d_loss= 1.019377, g_loss= 26.334724, D(X)= 0.417991, D(G(X))= 0.071838
Epoch: 2, d_loss= 0.846013, g_loss= 29.697008, D(X)= 0.716473, D(G(X))= 0.292667
Epoch: 2, d_loss= 1.210884, g_loss= 27.579756, D(X)= 0.347330, D(G(X))= 0.043484
Epoch: 2, d_loss= 0.973629, g_loss= 28.220411, D(X)= 0.523142, D(G(X))= 0.197574
Epoch: 2, d_loss= 1.399591, g_loss= 29.668671, D(X)= 0.917098, D(G(X))= 0.572206
Epoch: 2, d_loss= 0.995778, g_loss= 28.274496, D(X)= 0.847555, D(G(X))= 0.425578
Epoch: 2, d_loss= 0.913885, g_loss= 27.774822, D(X)= 0.471664, D(G(X))= 0.085932
Epoch: 2, d_loss= 1.313209, g_loss= 30.271637, D(X)= 0.905635, D(G(X))= 0.563485
Epoch: 2, d_loss= 1.071815, g_loss= 30.838984, D(X)= 0.750349, D(G(X))= 0.449669
Epoch: 2, d_loss= 0.934082, g_loss= 28.172203, D(X)= 0.684325, D(G(X))= 0.317157
Epoch: 2, d_loss= 1.334211, g_loss= 30.408672, D(X)= 0.850608, D(G(X))= 0.584276
Epoch: 2, d_loss= 0.772812, 

Epoch: 3, d_loss= 0.789555, g_loss= 28.208179, D(X)= 0.582501, D(G(X))= 0.111720
Epoch: 3, d_loss= 0.848351, g_loss= 28.148994, D(X)= 0.571657, D(G(X))= 0.159588
Epoch: 3, d_loss= 0.565030, g_loss= 29.358669, D(X)= 0.818914, D(G(X))= 0.146354
Epoch: 3, d_loss= 0.748315, g_loss= 28.026007, D(X)= 0.556273, D(G(X))= 0.054611
Epoch: 3, d_loss= 0.598229, g_loss= 28.676382, D(X)= 0.731210, D(G(X))= 0.136858
Epoch: 3, d_loss= 1.214043, g_loss= 26.272352, D(X)= 0.341191, D(G(X))= 0.079546
Epoch: 3, d_loss= 1.125094, g_loss= 28.925262, D(X)= 0.861584, D(G(X))= 0.497248
Epoch: 3, d_loss= 1.242530, g_loss= 30.137966, D(X)= 0.941784, D(G(X))= 0.529873
Epoch: 3, d_loss= 0.635634, g_loss= 28.116116, D(X)= 0.720127, D(G(X))= 0.147964
Epoch: 3, d_loss= 0.951987, g_loss= 29.291834, D(X)= 0.666739, D(G(X))= 0.322713
Epoch: 3, d_loss= 0.815768, g_loss= 27.466780, D(X)= 0.490244, D(G(X))= 0.041917
Epoch: 3, d_loss= 0.787481, g_loss= 28.317606, D(X)= 0.514227, D(G(X))= 0.030112
Epoch: 3, d_loss= 0.709737, 

Epoch: 4, d_loss= 0.932836, g_loss= 30.109337, D(X)= 0.919179, D(G(X))= 0.416767
Epoch: 4, d_loss= 1.243721, g_loss= 31.569021, D(X)= 0.962436, D(G(X))= 0.522308
Epoch: 4, d_loss= 0.653468, g_loss= 29.950989, D(X)= 0.803325, D(G(X))= 0.193615
Epoch: 4, d_loss= 0.636621, g_loss= 28.359583, D(X)= 0.702225, D(G(X))= 0.125240
Epoch: 4, d_loss= 0.655877, g_loss= 29.842421, D(X)= 0.853048, D(G(X))= 0.240729
Epoch: 4, d_loss= 0.652085, g_loss= 29.101498, D(X)= 0.682552, D(G(X))= 0.107205
Epoch: 4, d_loss= 0.500776, g_loss= 29.871624, D(X)= 0.862042, D(G(X))= 0.129687
Epoch: 4, d_loss= 0.917103, g_loss= 32.267437, D(X)= 0.955236, D(G(X))= 0.384379
Epoch: 4, d_loss= 0.563272, g_loss= 28.649128, D(X)= 0.789209, D(G(X))= 0.138661
Epoch: 4, d_loss= 1.040028, g_loss= 26.745510, D(X)= 0.415759, D(G(X))= 0.040870
Epoch: 4, d_loss= 0.985236, g_loss= 27.509274, D(X)= 0.433383, D(G(X))= 0.044759
Epoch: 4, d_loss= 1.018701, g_loss= 27.009066, D(X)= 0.401307, D(G(X))= 0.029176
Epoch: 4, d_loss= 0.523446, 

...

Epoch: 37, d_loss= 1.486142, g_loss= 28.815166, D(X)= 0.852916, D(G(X))= 0.591379
Epoch: 37, d_loss= 1.046845, g_loss= 28.406530, D(X)= 0.702260, D(G(X))= 0.324851
Epoch: 37, d_loss= 0.925677, g_loss= 29.042543, D(X)= 0.821460, D(G(X))= 0.352453
Epoch: 37, d_loss= 0.835540, g_loss= 29.146332, D(X)= 0.659591, D(G(X))= 0.178408
Epoch: 37, d_loss= 0.963629, g_loss= 29.881023, D(X)= 0.928432, D(G(X))= 0.351052
Epoch: 37, d_loss= 0.601023, g_loss= 29.340477, D(X)= 0.714061, D(G(X))= 0.061662
Epoch: 37, d_loss= 0.736058, g_loss= 28.652752, D(X)= 0.635544, D(G(X))= 0.060856
Epoch: 37, d_loss= 0.492574, g_loss= 29.658829, D(X)= 0.782150, D(G(X))= 0.043321
Epoch: 37, d_loss= 0.468806, g_loss= 31.408432, D(X)= 0.854991, D(G(X))= 0.085298
Epoch: 37, d_loss= 0.475126, g_loss= 31.102268, D(X)= 0.884480, D(G(X))= 0.080672
Epoch: 37, d_loss= 0.461004, g_loss= 30.561325, D(X)= 0.867085, D(G(X))= 0.067253
Epoch: 37, d_loss= 0.445027, g_loss= 32.004314, D(X)= 0.850121, D(G(X))= 0.048762
Epoch: 37, d_los

Epoch: 38, d_loss= 0.381347, g_loss= 31.408733, D(X)= 0.832202, D(G(X))= 0.011847
Epoch: 38, d_loss= 0.383630, g_loss= 30.739126, D(X)= 0.865102, D(G(X))= 0.029743
Epoch: 38, d_loss= 0.370039, g_loss= 31.235216, D(X)= 0.861575, D(G(X))= 0.014329
Epoch: 38, d_loss= 0.373973, g_loss= 33.033405, D(X)= 0.873321, D(G(X))= 0.011445
Epoch: 38, d_loss= 0.386792, g_loss= 32.764809, D(X)= 0.812367, D(G(X))= 0.006946
Epoch: 38, d_loss= 0.393048, g_loss= 30.269310, D(X)= 0.921906, D(G(X))= 0.043584
Epoch: 38, d_loss= 0.360326, g_loss= 31.617008, D(X)= 0.890664, D(G(X))= 0.017137
Epoch: 38, d_loss= 0.447618, g_loss= 31.180538, D(X)= 0.967733, D(G(X))= 0.037850
Epoch: 38, d_loss= 0.388144, g_loss= 31.659485, D(X)= 0.869694, D(G(X))= 0.017272
Epoch: 38, d_loss= 0.387762, g_loss= 31.351185, D(X)= 0.838584, D(G(X))= 0.017512
Epoch: 38, d_loss= 0.398262, g_loss= 30.995564, D(X)= 0.950852, D(G(X))= 0.032961
Epoch: 38, d_loss= 0.457717, g_loss= 30.977179, D(X)= 0.722947, D(G(X))= 0.006023
Epoch: 38, d_los

Epoch: 39, d_loss= 0.391302, g_loss= 32.217934, D(X)= 0.907341, D(G(X))= 0.029929
Epoch: 39, d_loss= 0.389586, g_loss= 31.393078, D(X)= 0.836836, D(G(X))= 0.013435
Epoch: 39, d_loss= 0.549621, g_loss= 32.519844, D(X)= 0.984973, D(G(X))= 0.080726
Epoch: 39, d_loss= 0.389836, g_loss= 32.852314, D(X)= 0.920682, D(G(X))= 0.016465
Epoch: 39, d_loss= 0.384768, g_loss= 30.910482, D(X)= 0.836837, D(G(X))= 0.012664
Epoch: 39, d_loss= 0.379357, g_loss= 33.484100, D(X)= 0.930880, D(G(X))= 0.014710
Epoch: 39, d_loss= 0.374731, g_loss= 31.376266, D(X)= 0.917667, D(G(X))= 0.019930
Epoch: 39, d_loss= 0.391018, g_loss= 31.890137, D(X)= 0.928603, D(G(X))= 0.038543
Epoch: 39, d_loss= 0.374365, g_loss= 32.594013, D(X)= 0.913181, D(G(X))= 0.022499
Epoch: 39, d_loss= 0.364459, g_loss= 31.021446, D(X)= 0.860489, D(G(X))= 0.011463
Epoch: 39, d_loss= 0.369885, g_loss= 32.247509, D(X)= 0.890262, D(G(X))= 0.014096
Epoch: 39, d_loss= 0.377760, g_loss= 32.651894, D(X)= 0.921585, D(G(X))= 0.017113
Epoch: 39, d_los

Epoch: 40, d_loss= 0.414708, g_loss= 33.010418, D(X)= 0.929664, D(G(X))= 0.043393
Epoch: 40, d_loss= 0.416102, g_loss= 31.750488, D(X)= 0.894799, D(G(X))= 0.046356
Epoch: 40, d_loss= 0.403190, g_loss= 31.106129, D(X)= 0.858927, D(G(X))= 0.027470
Epoch: 40, d_loss= 0.433032, g_loss= 29.415264, D(X)= 0.815150, D(G(X))= 0.035514
Epoch: 40, d_loss= 0.447358, g_loss= 30.479099, D(X)= 0.904077, D(G(X))= 0.076807
Epoch: 40, d_loss= 0.427523, g_loss= 31.455101, D(X)= 0.859411, D(G(X))= 0.036020
Epoch: 40, d_loss= 0.526458, g_loss= 28.809141, D(X)= 0.700676, D(G(X))= 0.014586
Epoch: 40, d_loss= 0.398172, g_loss= 32.277111, D(X)= 0.882640, D(G(X))= 0.034520
Epoch: 40, d_loss= 0.394367, g_loss= 31.976765, D(X)= 0.836931, D(G(X))= 0.020471
Epoch: 40, d_loss= 0.388511, g_loss= 29.737820, D(X)= 0.876099, D(G(X))= 0.033981
Epoch: 40, d_loss= 0.409935, g_loss= 32.461632, D(X)= 0.931781, D(G(X))= 0.043388
Epoch: 40, d_loss= 0.425932, g_loss= 31.702927, D(X)= 0.879948, D(G(X))= 0.057948
Epoch: 40, d_los

Epoch: 41, d_loss= 0.422091, g_loss= 30.890453, D(X)= 0.759633, D(G(X))= 0.007146
Epoch: 41, d_loss= 0.375061, g_loss= 31.520954, D(X)= 0.893493, D(G(X))= 0.025604
Epoch: 41, d_loss= 0.370910, g_loss= 32.717438, D(X)= 0.869335, D(G(X))= 0.009679
Epoch: 41, d_loss= 0.417636, g_loss= 32.188126, D(X)= 0.955371, D(G(X))= 0.044274
Epoch: 41, d_loss= 0.367382, g_loss= 31.139479, D(X)= 0.859669, D(G(X))= 0.009546
Epoch: 41, d_loss= 0.371804, g_loss= 31.900032, D(X)= 0.911374, D(G(X))= 0.017644
Epoch: 41, d_loss= 0.431794, g_loss= 32.231697, D(X)= 0.952003, D(G(X))= 0.052557
Epoch: 41, d_loss= 0.556319, g_loss= 29.200645, D(X)= 0.632810, D(G(X))= 0.004999
Epoch: 41, d_loss= 0.380297, g_loss= 31.799091, D(X)= 0.887828, D(G(X))= 0.013316
Epoch: 41, d_loss= 0.364275, g_loss= 32.653786, D(X)= 0.882813, D(G(X))= 0.010059
Epoch: 41, d_loss= 0.388369, g_loss= 29.768215, D(X)= 0.863350, D(G(X))= 0.022594
Epoch: 41, d_loss= 0.374018, g_loss= 31.306059, D(X)= 0.911173, D(G(X))= 0.021504
Epoch: 41, d_los